# SQL Lite Parsing and Processing

## SQL Lite data creation

In [13]:
import sqlite3
import os
from typing import List, Any, Dict
from langchain_core.documents import Document

In [ ]:

os.makedirs('data/databases', exist_ok=True)

In [2]:
## Create a sample SQLite database
conn = sqlite3.connect('data/databases/company.db')
cursor = conn.cursor()

In [4]:
# Create table - employees
cursor.execute(''' CREATE TABLE IF NOT EXISTS employees
               (id INTEGER PRIMARY KEY, name TEXT, role TEXT, department TEXT, salary REAL)''')

In [5]:
# Create table - projects
cursor.execute(''' CREATE TABLE IF NOT EXISTS projects
               (id INTEGER PRIMARY KEY, name TEXT, status TEXT, budget REAL, lead_id INTEGER)''')

In [6]:
# Insert Sample data into employees and projects tables
employees = [
    (1, 'John Doe', 'Software Engineer', 'IT', 95000),
    (2, 'Jane Smith', 'Data Scientist', 'IT', 100000),   
    (3, 'Emily Davis', 'Product Manager', 'Product', 110000),
    (4, 'Michael Brown', 'Designer', 'Design', 98000),
    (5, 'Sarah Wilson', 'HR Manager', 'HR', 105000)
]

projects = [
    (1, 'RAG Implementation', 'Active', 150000, 1),
    (2, 'Data Pipeline Upgrade', 'Completed', 120000, 2),
    (3, 'Custom Portal', 'Planning', 200000, 3),
    (4, 'ML platform', 'Active', 250000, 5),
    (5, 'ML Model Deployment', 'Completed', 180000, 4)
]

In [7]:
cursor.executemany('INSERT OR REPLACE INTO employees VALUES(?,?,?,?,?)', employees)
cursor.executemany('INSERT OR REPLACE INTO projects VALUES(?,?,?,?,?)', projects)

In [8]:
cursor.execute('SELECT * FROM employees')

In [10]:
conn.commit()
conn.close()

## Database content extraction

In [11]:
from langchain_community.utilities import SQLDatabase
from langchain_community.document_loaders import SQLDatabaseLoader

/Users/neeladnatarajan/DSProjects/LLMOps/hw/RAGUdemy/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# Method 1: SQLDatabase Utility
db = SQLDatabase.from_uri("sqlite:///data/databases/company.db")

# Get Database info
print(f"Tables: {db.get_usable_table_names()}")
print(f"\nTable DDL:")
print(db.get_table_info())

Tables: ['employees', 'projects']

Table DDL:

CREATE TABLE employees (
	id INTEGER, 
	name TEXT, 
	role TEXT, 
	department TEXT, 
	salary REAL, 
	PRIMARY KEY (id)
)

/*
3 rows from employees table:
id	name	role	department	salary
1	John Doe	Software Engineer	IT	95000.0
2	Jane Smith	Data Scientist	IT	100000.0
3	Emily Davis	Product Manager	Product	110000.0
*/


CREATE TABLE projects (
	id INTEGER, 
	name TEXT, 
	status TEXT, 
	budget REAL, 
	lead_id INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from projects table:
id	name	status	budget	lead_id
1	RAG Implementation	Active	150000.0	1
2	Data Pipeline Upgrade	Completed	120000.0	2
3	Custom Portal	Planning	200000.0	3
*/


In [20]:
# Method 2: Custom SQL to document creation
print(f"\n Custom SQL to document creation...\n")

def sql_to_documents(db_path: str) -> List[Document]:
    """ Convert SQLITE3 SQL Databases into Document"""
    conn = sqlite3.connect(db_path)
    cursor=conn.cursor()
    documents=[]

    # Strategy 1: Create documents from each table
    cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
    tables = cursor.fetchall()

    # Iterating through each table
    for table in tables:
        table_name = table[0]

        # Get table schema
        cursor.execute(f"PRAGMA table_info({table_name})")
        columns=cursor.fetchall()
        column_names=[col[1] for col in columns]

        # Get table data
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Create table overview
        table_content = f"Table: {table_name}\n"
        table_content += f"Columns: {', '.join(column_names)}\n"
        table_content += f"Total Records: {len(rows)}\n"

        # Add sample records
        table_content += "Sample Records\n"
        for row in rows[:5]:
            record = dict(zip(column_names, row))
            table_content += f"{record}\n"
        
        doc = Document(
            page_content=table_content,
            metadata={
                'source': db_path,
                'table_name': table_name,
                'num_records': len(rows),
                'data_type': 'sql_table'

            }
        )
        documents.append(doc)
        
    # Strategy 2: Create relationship documents
    # Example Join Employees and Projects
    cursor.execute("""
    SELECT e.name, e.role, p.name as project_name, p.status 
    FROM employees e
    JOIN projects p ON e.id = p.lead_id
    """)
    relationship = cursor.fetchall()
    rel_content = "Employee-Project Relationships:\n\n"
    for rel in relationship:
        rel_content += f"{rel[0]} ({rel[1]}) leads {rel[2]} - status: {rel[3]}\n"
    rel_doc = Document(
        page_content=rel_content,
        metadata={
            'source': db_path,
            'data_type': 'sql_relationships',
            'query': 'employee-project-join'
        }
    )
    documents.append(rel_doc)

    return documents
    conn.close()


 Custom SQL to document creation...



In [21]:
sql_to_documents("data/databases/company.db")

[Document(metadata={'source': 'data/databases/company.db', 'table_name': 'employees', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table: employees\nColumns: id, name, role, department, salary\nTotal Records: 5\nSample Records\n{'id': 1, 'name': 'John Doe', 'role': 'Software Engineer', 'department': 'IT', 'salary': 95000.0}\n{'id': 2, 'name': 'Jane Smith', 'role': 'Data Scientist', 'department': 'IT', 'salary': 100000.0}\n{'id': 3, 'name': 'Emily Davis', 'role': 'Product Manager', 'department': 'Product', 'salary': 110000.0}\n{'id': 4, 'name': 'Michael Brown', 'role': 'Designer', 'department': 'Design', 'salary': 98000.0}\n{'id': 5, 'name': 'Sarah Wilson', 'role': 'HR Manager', 'department': 'HR', 'salary': 105000.0}\n"),
 Document(metadata={'source': 'data/databases/company.db', 'table_name': 'projects', 'num_records': 5, 'data_type': 'sql_table'}, page_content="Table: projects\nColumns: id, name, status, budget, lead_id\nTotal Records: 5\nSample Records\n{'id': 1, 'name